### 1. Importación de Librerias

In [1]:
#Importación de librerías Requeridas
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import numpy as np
import sys
import tensorflow as tf
import time
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight

### 2. Lectura del data frame

In [2]:
file_path = 'banklimpio.csv'
df = pd.read_csv(file_path, sep=',')
df.shape

(77473, 84)

### 3. 

In [ ]:
#Normalizar todas las capas
